In [ ]:
import keras
from keras.layers import Input, Dense
from keras.models import Model

In [ ]:
# Define input shape
input_shape = (784,)

# Define encoding dimension
encoding_dim = 32

# Define input layer
input_layer = Input(shape=input_shape)

In [ ]:
# Define encoder layers
encoded_layer1 = Dense(128, activation='relu')(input_layer)
encoded_layer2 = Dense(64, activation='relu')(encoded_layer1)
encoded_layer3 = Dense(encoding_dim, activation='relu')(encoded_layer2)

# Define decoder layers
decoded_layer1 = Dense(64, activation='relu')(encoded_layer3)
decoded_layer2 = Dense(128, activation='relu')(decoded_layer1)
decoded_layer3 = Dense(784, activation='sigmoid')(decoded_layer2)

In [ ]:
# Define autoencoder model
autoencoder_model = Model(inputs=input_layer, outputs=decoded_layer3)

# Define encoder model
encoder_model = Model(inputs=input_layer, outputs=encoded_layer3)

# Define decoder model
decoder_input = Input(shape=(encoding_dim,))
decoder_layer1 = autoencoder_model.layers[-3](decoder_input)
decoder_layer2 = autoencoder_model.layers[-2](decoder_layer1)
decoder_layer3 = autoencoder_model.layers[-1](decoder_layer2)
decoder_model = Model(inputs=decoder_input, outputs=decoder_layer3)

# Compile autoencoder model
autoencoder_model.compile(optimizer='adam', loss='binary_crossentropy')

# Load data and normalize to range [0, 1]
(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

# Train autoencoder model
autoencoder_model.fit(x_train, x_train,
                      epochs=50,
                      batch_size=256,
                      shuffle=True,
                      validation_data=(x_test, x_test))

# Use encoder and decoder models to encode and decode data
encoded_data = encoder_model.predict(x_test)
decoded_data = decoder_model.predict(encoded_data)